<a href="https://colab.research.google.com/github/letizia-z/neuroscience-sql-student-project/blob/main/neuroscience_sql_student_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Surname | GitHub |
| :---        |    :---  |
| *Sirghe A.* | *https://github.com/sandasirghe*   |
| *Zanetti L.* | *https://github.com/letizia-z* |


# **NEUROSCIENTIFIC INVESTIGATION: Database of Studies on the Brain's Anatomical-Functional Correlations**

## **0. Declaration of Originality**
The project was developed entirely by Sirghe A. and Zanetti L.

Responsibility for the content is equally shared between the two. The only tool based on text generation used during the development of the project was ChatGPT, employed solely for gathering information for the structuring and population of the database.



|Other useful resources|URL|
| :---        |    :---  |
|*DuckDB_in_Jupyter_Notebooks*|*https://colab.research.google.com/drive/1eOA2FYHqEfZWLYssbUxdIpSL3PFxWVjk?usp=sharing#scrollTo=CvY8OgfV3ckB*|
|*Excel Import - DuckDB* |*https://duckdb.org/docs/guides/file_formats/excel_import.html* |
|*ALTER TABLE Statement* |*https://duckdb.org/docs/sql/statements/alter_table.html#drop-column* |
|*How to Insert a Value that Contains an Apostrophe in PostgreSQL?* |*https://www.geeksforgeeks.org/how-to-insert-a-value-that-contains-an-apostrophe-in-postgresql/* |
|*UPDATE Statement* |*https://duckdb.org/docs/sql/statements/update.html* |
|*PostgreSQL CONCAT() Function* |*https://neon.tech/postgresql/postgresql-string-functions/postgresql-concat-function* |
|*PostgreSQL Primary Key* |*https://neon.tech/postgresql/postgresql-tutorial/postgresql-primary-key* |
|*PostgreSQL 9.7 Pattern Matching* |*https://www.postgresql.org/docs/current/functions-matching.html#POSIX-SYNTAX-DETAILS* |




---


## **1. Project Description**
The goal is to design a **database to systematically collect scientific articles** documenting experiments or studies on the relationship between specific areas of the brain and various brain functions (such as memory, attention, and language).
The results are based on the use of specific brain activity stimulation or recording techniques (e.g., fMRI, PET, EEG), which differ in their precision, defined by their spatial and temporal resolution.


## *1.1 ER-model*

<img alt="png" src="https://raw.githubusercontent.com/letizia-z/neuroscience-sql-student-project/refs/heads/main/ER_sql.png" width="75%">

![svg](https://raw.githubusercontent.com/letizia-z/neuroscience-sql-student-project/refs/heads/main/ER_sql.svg)

## *1.2 Relational Model*
* study (*study_id*, publication_date, doi, technique_abbreviation)
* study_title (study_id, *title*)
* written_by (*study_id*, *author_id*)
* author (*author_id*, surname, orcid)
* technique (*abbreviation*, space_resolution, time_resolution)
* cerebral_function (*functional_macroarea*, *specification*, *brain_area*, *study_id*)
* cerebral_area(*area_name*, main_structure, lobe, emisphere)

***Integrity Constraints:***
* study_title.study_id -> study.study_id
* study.technique_abbreviation -> technique.abbreviation
* written_by.study_id -> study.study_id
* written_by.author_id -> author.author_id
* cerebral_function.study_id -> study.study_id
* cerebral_function.brain_area -> cerebral_area.area_name





---


# ***2. Implementation***
To create the Notebook, we decided to use **DuckDB**, [based on the PostgreSQL dialect](https://duckdb.org/docs/sql/introduction.html), allowing us to work with Excel files, which in this case we import from GitHub.

## *2.1 Preparation*

Firs of all, download [data.xlsx](https://github.com/letizia-z/neuroscience-sql-student-project/blob/main/data.xlsx/) from the GitHub repository, and upload it in the current notebook.

We install and import the necessary libraries using the `--quiet` flag to reduce output messages during installation:
* **DuckDB**: mainly used to manage an in-memory analytical database (`%sql duckdb:///:memory:` according to the [DuckDB documentation](https://duckdb.org/docs/connect/overview.html), allows for temporary in-memory data storage without needing to save it to disk)
* **Duckdb-engine**: enables integration of DuckDB with other applications, such as working with Python
* **Jupysql**: allows SQL commands to be written directly in the notebook, with clearer visualization
* **Pandas**: a useful library for data analysis, especially when working with tables and databases


In [ ]:
!pip install --quiet duckdb
!pip install --quiet duckdb-engine
!pip install --quiet jupysql
!pip install --quiet pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 9.2 MB/s eta 0:00:00


In [ ]:
import duckdb
import pandas as pd

# Jupysql extension to create SQL cells, instead of working with Python script
%load_ext sql

In [ ]:
%config SqlMagic.autopandas = True    # Query results are automatically converted into pandas DataFrames
%config SqlMagic.feedback = False     # Disables feedback messages (e.g., 'Query executed successfully')
%config SqlMagic.displaycon = False   # Disables database connection information

In [ ]:
%sql duckdb:///:memory:     -- Connect Duckdb to the main memory
# %sql duckdb:///myfile.db  # To save the DB in files, but we must remember to download it

Finally, we install `spatial`, a specific DuckDB extension designed to enable geospatial functionalities in the database (e.g., creating coordinates and calculating distances). In our case, however, we use it to analyze individual Excel sheets grouped within a single file, [Dati.xlsx](https://docs.google.com/spreadsheets/d/1Kqzk0il1MsXUnckepoBA05uIvlBaYoHS/edit?usp=drive_link&ouid=111941362116965165185&rtpof=true&sd=true), as recommended by the [official documentation](https://duckdb.org/docs/guides/file_formats/excel_import.html).

> *Note: While creating the queries, we encountered issues reading the table `cerebral_area`. Assuming it was related to a peculiar EOF parsing issue, an extra sheet was added to `Dati.xlsx` to bypass the problem and allow proper interpretation of the table.*



In [ ]:
%%sql
INSTALL spatial;
LOAD spatial;
-- to read Excel

,Success


In [ ]:
# Test query, to check whether Spatial is actually reading the .xlsx file or not
%%sql
SELECT * FROM st_read('data.xlsx',
layer = 'study_title',
open_options = ['HEADERS=FORCE'])
-- Layer and open_options are necessary, in that sql uses a "made up" header, inserting "study_id" and "title" in line 0

,study_id,title
0,s001,Exercise training increases size of hippocampu...
1,s002,The time course of feature-selective attention...
2,s003,Enhancing cognition using transcranial electri...
3,s004,Placebo-induced changes in fMRI in the anticip...
4,s005,Cortisol awakening response prompts dynamic re...
5,s006,Neuroimaging studies of working memory: a meta...
6,s007,The emerging field of non-invasive brain stimu...
7,s008,Attentional Modulation in Early Visual Cortex:...


## *2.2 Query*

### 2.2.1 Queries by Study and Author

In [ ]:
# 2. Find the most recent studies
# 2.1 With title
%%sql
SELECT study_id, title, publication_date
FROM st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE']) AS study
     NATURAL JOIN st_read('data.xlsx', layer = 'study_title', open_options = ['HEADERS=FORCE']) AS study_title
ORDER BY study.publication_date DESC;

,study_id,title,publication_date
0,s007,The emerging field of non-invasive brain stimu...,2024-12-03
1,s005,Cortisol awakening response prompts dynamic re...,2024-03-22
2,s002,The time course of feature-selective attention...,2024-02-28
3,s008,Attentional Modulation in Early Visual Cortex:...,2024-01-01
4,s003,Enhancing cognition using transcranial electri...,2015-06-03
5,s001,Exercise training increases size of hippocampu...,2011-02-15
6,s004,Placebo-induced changes in fMRI in the anticip...,2004-02-20
7,s006,Neuroimaging studies of working memory: a meta...,2003-12-03


In [ ]:
# 2.2 With cerebral functions
%%sql
WITH study_info AS (
SELECT study_id, publication_date
FROM st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE']) AS study
     NATURAL JOIN st_read('data.xlsx', layer = 'study_title', open_options = ['HEADERS=FORCE']) AS study_title)

SELECT study_info.study_id, functional_macroarea, specification, publication_date
FROM study_info
     JOIN st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE']) AS cerebral_function
     ON study_info.study_id = cerebral_function.study_id
WHERE functional_macroarea = 'Attention'
ORDER BY study_info.publication_date DESC;

,study_id,functional_macroarea,specification,publication_date
0,s002,Attention,Selective,2024-02-28
1,s002,Attention,Spatial,2024-02-28
2,s008,Attention,Selective,2024-01-01
3,s008,Attention,General,2024-01-01
4,s003,Attention,General,2015-06-03
5,s004,Attention,Selective,2004-02-20
6,s006,Attention,Selective,2003-12-03


In [ ]:
# 3. Find every author who worked on a certain study
%%sql
SELECT written_by.study_id, written_by.author_id, author.surname, author.orcid
FROM st_read('data.xlsx', layer = 'author', open_options = ['HEADERS=FORCE']) AS author,
     st_read('data.xlsx', layer = 'written_by', open_options = ['HEADERS=FORCE']) AS written_by
WHERE written_by.author_id = author.author_id AND study_id = 's004';

,study_id,author_id,surname,orcid
0,s004,a027,Wager,0000-0002-1936-5574
1,s004,a028,Rilling,0000-0002-6980-6556
2,s004,a029,Smith,None
3,s004,a030,Sokolik,None
4,s004,a031,Casey,None
5,s004,a032,Davidson,0000-0002-3033-3912
6,s004,a033,Kosslyn,0000-0002-7465-944X
7,s004,a034,Rose,None
8,s004,a035,Cohen,0000-0002-4891-5160


In [ ]:
# 4. Find all the studies on which a certain author has worked on
%%sql
SELECT written_by.study_id, written_by.author_id, author.surname, author.orcid
FROM st_read('data.xlsx', layer = 'author', open_options = ['HEADERS=FORCE']) AS author,
     st_read('data.xlsx', layer = 'written_by', open_options = ['HEADERS=FORCE']) AS written_by
WHERE written_by.author_id = author.author_id AND author.author_id = 'a027';

,study_id,author_id,surname,orcid
0,s006,a027,Wager,0000-0002-1936-5574
1,s004,a027,Wager,0000-0002-1936-5574


In [ ]:
# 5. Find all the authors by their intital and the studies they worked on
%%sql
SELECT surname, written_by.author_id, written_by.study_id
FROM st_read('data.xlsx', layer = 'author', open_options = ['HEADERS=FORCE']) AS author JOIN
     st_read('data.xlsx', layer = 'written_by', open_options = ['HEADERS=FORCE']) AS written_by USING (author_id)
WHERE surname LIKE 'C%';

,surname,author_id,study_id
0,Chaddock,a006,s001
1,Cohen Kadosh,a025,s003
2,Casey,a031,s004
3,Cohen,a035,s004
4,Chen,a040,s005
5,Casula,a047,s007


In [ ]:
# 6. Find the number of authors who worked on each study
%%sql
SELECT study_id, COUNT(DISTINCT author_id) AS authors_number
FROM st_read('data.xlsx', layer = 'written_by', open_options = ['HEADERS=FORCE'])
GROUP BY study_id;

,study_id,authors_number
0,s002,2
1,s004,9
2,s007,9
3,s008,2
4,s001,18
5,s003,6
6,s005,7
7,s006,2


In [ ]:
# 7. Find all authors who researched a certain cerebral function
%%sql
WITH researchers AS (
    SELECT written_by.author_id, written_by.study_id, author.surname, author.orcid
    FROM st_read('data.xlsx', layer = 'author', open_options = ['HEADERS=FORCE']) AS author,
         st_read('data.xlsx', layer = 'written_by', open_options = ['HEADERS=FORCE']) AS written_by
    WHERE written_by.author_id = author.author_id)

SELECT researchers.surname, researchers.orcid, cerebral_function.functional_macroarea, cerebral_function.specification
FROM researchers
    JOIN st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE']) AS cerebral_function
     ON researchers.study_id = cerebral_function.study_id
WHERE cerebral_function.functional_macroarea = 'Memory'
ORDER BY researchers.surname

,surname,orcid,functional_macroarea,specification
0,Alves,0000-0002-6102-7616,Memory,Spatial
1,Basak,0000-0002-0364-0324,Memory,Spatial
2,Chaddock,None,Memory,Spatial
3,Erickson,0000-0001-8736-981X,Memory,Spatial
4,Heo,None,Memory,Spatial
5,Kim,None,Memory,Spatial
6,Kramer,0000-0001-5870-2724,Memory,Spatial
7,Mailey,0000-0001-7672-445X,Memory,Spatial
8,Martin,None,Memory,Spatial
9,McAuley,None,Memory,Spatial


In [ ]:
# 8. Find all studies who used a certain technique
%%sql
SELECT study_id, technique_abbreviation
FROM st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE'])
WHERE technique_abbreviation = 'fMRI'

,study_id,technique_abbreviation
0,s002,fMRI
1,s004,fMRI


### 2.2.2 Queries by Cerebral Function and Area

In [ ]:
# 9. Identify which cognitive functions can be performed by each brain area in the database
%%sql
SELECT DISTINCT cerebral_area.lobe, cerebral_area.main_structure, cerebral_area.area_name, cerebral_function.functional_macroarea, cerebral_function.specification
FROM st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE']) AS cerebral_function
  JOIN st_read('data.xlsx', layer = 'cerebral_area', open_options = ['HEADERS=FORCE']) AS cerebral_area
  ON cerebral_area.area_name = cerebral_function.brain_area
ORDER BY cerebral_area.lobe, cerebral_area.main_structure, cerebral_function.functional_macroarea

,lobe,main_structure,area_name,functional_macroarea,specification
0,Frontal,Cerebral cortex,Prefrontal cortex,Attention,General
1,Frontal,Cerebral cortex,Prefrontal cortex,Cognitive functions,General
2,Frontal,Cerebral cortex,Prefrontal cortex,Emotions,Regulation
3,Frontal,Cerebral cortex,Prefrontal cortex,Executive functions,Planning
4,Frontal,Cerebral cortex,Prefrontal cortex,Executive functions,General
5,Frontal,Cerebral cortex,Prefrontal cortex,Memory,General
6,Frontal,Limbic system,Anterior cingulate cortex,Attention,Selective
7,Frontal,Limbic system,Anterior cingulate cortex,Emotions,Regulation
8,Occipital,Cerebral cortex,Visual association cortex,Attention,Spatial
9,Temporal,Limbic system,Hippocampus,Memory,General


In [ ]:
# 9.1 Identify all the functions that can be performed by a certain brain area
%%sql
SELECT DISTINCT brain_area, functional_macroarea, specification
FROM st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE'])
WHERE brain_area = 'Hippocampus'

,brain_area,functional_macroarea,specification
0,Hippocampus,Memory,Spatial
1,Hippocampus,Perception,Recognition
2,Hippocampus,Memory,General


In [ ]:
# 9.2 Find all the areas of the brain needed to perform a certain cognitive function (macroarea)
%%sql
SELECT DISTINCT functional_macroarea, brain_area
FROM st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE'] )
WHERE functional_macroarea = 'Memory'

,functional_macroarea,brain_area
0,Memory,Hippocampus
1,Memory,Prefrontal cortex


In [ ]:
# 9.3 Count the number of functional macroareas for each cerebral area
%%sql
SELECT DISTINCT brain_area, COUNT(DISTINCT functional_macroarea) AS number_of_functions
FROM st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE'])
GROUP BY brain_area
ORDER BY number_of_functions DESC

,brain_area,number_of_functions
0,Prefrontal cortex,5
1,Hippocampus,2
2,Anterior cingulate cortex,2
3,Visual association cortex,1


In [ ]:
# 10. Ranking of the most used techniques
%%sql
SELECT technique_abbreviation, COUNT(technique_abbreviation) AS num
FROM st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE']) AS study
GROUP BY technique_abbreviation
ORDER BY num DESC;

,technique_abbreviation,num
0,fMRI,2
1,tDCS,2
2,MRI,2
3,ERP,1
4,SSVEP,1


In [ ]:
# 10.1 Identify which techniques are most used to study a certain cognitive function
%%sql
SELECT cerebral_function.functional_macroarea, study.technique_abbreviation, COUNT(study.technique_abbreviation) AS num
FROM st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE']) AS cerebral_function
JOIN st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE']) AS study
ON study.study_id = cerebral_function.study_id -- here we could've also used a NATURAL JOIN
GROUP BY study.technique_abbreviation, cerebral_function.functional_macroarea
ORDER BY cerebral_function.functional_macroarea, num DESC;

,functional_macroarea,technique_abbreviation,num
0,Attention,fMRI,3
1,Attention,SSVEP,2
2,Attention,tDCS,1
3,Attention,ERP,1
4,Cognitive functions,tDCS,1
5,Emotions,fMRI,3
6,Emotions,MRI,2
7,Executive functions,tDCS,2
8,Executive functions,fMRI,1
9,Executive functions,MRI,1


In [ ]:
# 10.2 Find which techniques are most used to study a conitive functions and its main brain area
%%sql
SELECT study.technique_abbreviation, COUNT(study.technique_abbreviation) AS num, cerebral_function.functional_macroarea, cerebral_function.brain_area
FROM st_read('data.xlsx', layer = 'cerebral_function', open_options = ['HEADERS=FORCE']) AS cerebral_function
  NATURAL JOIN st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE']) AS study
WHERE cerebral_function.functional_macroarea = 'Emotions' AND cerebral_function.brain_area = 'Anterior cingulate cortex'
GROUP BY study.technique_abbreviation, cerebral_function.functional_macroarea, cerebral_function.brain_area
ORDER BY num DESC, technique_abbreviation

,technique_abbreviation,num,functional_macroarea,brain_area
0,fMRI,2,Emotions,Anterior cingulate cortex
1,MRI,1,Emotions,Anterior cingulate cortex


In [ ]:
# 11. Identify which techniques have not been used yet
%%sql
SELECT abbreviation
FROM st_read('data.xlsx', layer = 'technique', open_options = ['HEADERS=FORCE']) AS technique
WHERE abbreviation NOT IN (SELECT technique_abbreviation
                    FROM st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE']) AS study
);

,abbreviation
0,sMRI
1,PET
2,SPECT
3,DTI
4,EEG
5,MEG
6,ECoG
7,NIRS
8,TMS
9,MRS


In [ ]:
#11.1 Full list of techniques and whether they have been used in a study
%%sql
SELECT abbreviation, study_id
FROM st_read('data.xlsx', layer = 'technique', open_options = ['HEADERS=FORCE']) AS technique
     LEFT OUTER JOIN st_read('data.xlsx', layer = 'study', open_options = ['HEADERS=FORCE']) AS study
		 ON technique_abbreviation = abbreviation;

,abbreviation,study_id
0,MRI,s005
1,fMRI,s004
2,ERP,s006
3,tDCS,s007
4,SSVEP,s008
5,MRI,s001
6,fMRI,s002
7,tDCS,s003
8,sMRI,None
9,PET,None


# ***3. Additional DDL (ALTER-ADD-DROP) and DML (UPDATE) Operations***

In this final section of the Notebook, we decided to include some DDL and DML operations:
* The first is a simple deletion of a column from the "cerebral_area" table that we found to be redundant *(see 3.1)*;
* The second was meant to be a series of steps to obtain a custom surrogate key similar to those already present in the Excel tables queried in the previous block *(see 3.2)*.

After several attempts, we found it difficult to achieve this with our current knowledge. Nevertheless, we documented the processes we followed and decided not to remove them from the Notebook, as they are practical examples of operations that test the flexibility limits of both databases in general and DuckDB in particular.

### *3.1 Deletion of the "emisphere" column from the "cerebral_area" table*

Since we couldn't find any indication that DuckDB allows modifying (and not just reading) Excel tables, it was necessary to create a new table, "cerebral_area".
As this is just a demonstration, we only partially populated it (a simple copy-paste of the SQL code from the project report was sufficient).

In [ ]:
%%sql
CREATE TABLE cerebral_area(
  area_name varchar(35) PRIMARY KEY,
  main_structure varchar(35),
  emisphere varchar(8),
  lobe varchar(15),
  CONSTRAINT emisphere_values
             CHECK(emisphere IN ('Right', 'Left'))
);



INSERT INTO cerebral_area (area_name, main_structure, emisphere, lobe)
VALUES
	('Amygdala', 'Limbic system', 'Right', 'Temporal'),
	('Broca Area', 'Cerebral cortex', 'Left', 'Frontal'),
	('Wernicke Area', 'Cerebral cortex', 'Left', 'Temporal'),
	('Olfactory Bulb', 'Limbic system', NULL, NULL),
	('Cerebellum', 'Cerebellum', NULL, NULL),
  ('Motor association cortex', 'Cerebral cortex', NULL, 'Frontal'),
  ('Somatosensory association cortex', 'Cerebral cortex', NULL, 'Parietal'),
	('Auditory association cortex', 'Cerebral cortex', NULL, 'Temporal'),
  ('Visual association cortex', 'Cerebral cortex', NULL, 'Occipital'),
	('Primary sensory cortex', 'Cerebral cortex', NULL, 'Parietal'),
  ('Anterior cingulate cortex', 'Limbic system', NULL, 'Frontal'),
  ('Primary motor cortex', 'Cerebral cortex', NULL, 'Frontal'),
  ('Orbitofrontal cortex', 'Cerebral cortex', NULL, 'Frontal'),
  ('Prefrontal cortex', 'Cerebral cortex', NULL, 'Frontal'),
  ('Somatosensory cortex', 'Cerebral cortex', NULL, 'Parietal'),
  ('Primary somatosensory cortex', 'Cerebral cortex', NULL, 'Parietal'),
  ('Primary visual cortex', 'Cerebral cortex', NULL, 'Occipital'),
  ('Middle temoral gyrus', 'Cerebral cortex', 'Left', 'Temporal'),
	('Basal ganglia', 'Basal ganglia', NULL, NULL),
	('Insula', 'Cerebral cortex', NULL, 'Temporal'),
	('Thalamus', 'Thalamus', NULL, NULL),
	('Hippocampus', 'Limbic system', NULL, 'Temporal');



,Success


In [ ]:
# Output of the original table
%%sql
SELECT *
FROM cerebral_area;

,area_name,main_structure,emisphere,lobe
0,Amygdala,Limbic system,Right,Temporal
1,Broca Area,Cerebral cortex,Left,Frontal
2,Wernicke Area,Cerebral cortex,Left,Temporal
3,Olfactory Bulb,Limbic system,None,None
4,Cerebellum,Cerebellum,None,None
5,Motor association cortex,Cerebral cortex,None,Frontal
6,Somatosensory association cortex,Cerebral cortex,None,Parietal
7,Auditory association cortex,Cerebral cortex,None,Temporal
8,Visual association cortex,Cerebral cortex,None,Occipital
9,Primary sensory cortex,Cerebral cortex,None,Parietal


In [ ]:
# ALTER TABLE and DROP operation to delete the "emisphere" column
%%sql
ALTER TABLE cerebral_area DROP emisphere;

,Success


In [ ]:
# Visualization of the table after the DROP
%%sql
SELECT *
FROM cerebral_area;

,area_name,main_structure,lobe
0,Amygdala,Limbic system,Temporal
1,Broca Area,Cerebral cortex,Frontal
2,Wernicke Area,Cerebral cortex,Temporal
3,Olfactory Bulb,Limbic system,None
4,Cerebellum,Cerebellum,None
5,Motor association cortex,Cerebral cortex,Frontal
6,Somatosensory association cortex,Cerebral cortex,Parietal
7,Auditory association cortex,Cerebral cortex,Temporal
8,Visual association cortex,Cerebral cortex,Occipital
9,Primary sensory cortex,Cerebral cortex,Parietal


## *3.2 Modifying a numeric surrogate key to support letter-number combinations*

Finally, we wanted to try replicating the format of the keys like "study_id" (`s00n`) and "author_id" (`a00n`) using a numerically generated surrogate key from the software.

The general idea was to create a table where the initial ID is generated by DuckDB using `CREATE SEQUENCE`, and then modified through DDL operations (`ALTER TABLE`) to achieve the desired result.

The first method attempted was to create a column "partial_id" with a default value of "a00", which was then concatenated (after converting the generated number from INTEGER to string) to the software-generated number.

> *Note: We are aware that there are more effective and less cumbersome methods for achieving good results, as shown in [this article](https://www.sqlteam.com/articles/custom-auto-generated-sequences-with-sql-server). We're also aware of the need to reflect on one's actual requirements even before implementing such a key, even if the tools are well understood.*

In [ ]:
%%sql
CREATE SEQUENCE study_id_seq START 1; -- We create a generic sequence which is going to be substituted by a concatenation

CREATE TABLE study (
    study_id INTEGER DEFAULT nextval('study_id_seq'),
    doi VARCHAR(50) UNIQUE,
    publication_date DATE,
    technique_abbreviation VARCHAR(5),
    --CONSTRAINT study_id_as_pkey -- The name of the CONSTRAINT, while being optional, will prove to be useful later on
    --PRIMARY KEY (study_id)
);

INSERT INTO study (doi, publication_date, technique_abbreviation)
VALUES
      ('10.1073/pnas.1015950108', '2011-02-15', 'MRI'), -- Being a surrogate key, adding the study_id was unnecessary
      ('10.1073/pnas.2309975121', '2024-02-28', 'fMRI'),
      ('10.1016/j.cobeha.2015.06.003', '2015-06-03', 'tDCS'),
      ('10.1126/science.1093065', '2004-02-20', 'fMRI'),
      ('10.1073/pnas.2405850121', '2024-03-22', 'MRI'),
      ('10.3758/cabn.3.4.255', '2003-12-03', 'ERP'),
      ('10.1093/brain/awae292', '2024-12-03', 'tDCS'),
      ('10.1162/jocn_a_02070', '2024-01-01', 'SSVEP');


,Success


In [ ]:
# Visualization of the table
%%sql
SELECT *
FROM study;

,study_id,doi,publication_date,technique_abbreviation
0,1,10.1073/pnas.1015950108,2011-02-15,MRI
1,2,10.1073/pnas.2309975121,2024-02-28,fMRI
2,3,10.1016/j.cobeha.2015.06.003,2015-06-03,tDCS
3,4,10.1126/science.1093065,2004-02-20,fMRI
4,5,10.1073/pnas.2405850121,2024-03-22,MRI
5,6,10.3758/cabn.3.4.255,2003-12-03,ERP
6,7,10.1093/brain/awae292,2024-12-03,tDCS
7,8,10.1162/jocn_a_02070,2024-01-01,SSVEP


In [ ]:
# Adding the column with the value 'a00' to concatenat to the number of study_id
%%sql
ALTER TABLE study ADD COLUMN partial_id CHAR(3) DEFAULT 'a00';

,Success


In [ ]:
# Visualization to check changes
%%sql
SELECT *
FROM study;

,study_id,doi,publication_date,technique_abbreviation,partial_id
0,1,10.1073/pnas.1015950108,2011-02-15,MRI,a00
1,2,10.1073/pnas.2309975121,2024-02-28,fMRI,a00
2,3,10.1016/j.cobeha.2015.06.003,2015-06-03,tDCS,a00
3,4,10.1126/science.1093065,2004-02-20,fMRI,a00
4,5,10.1073/pnas.2405850121,2024-03-22,MRI,a00
5,6,10.3758/cabn.3.4.255,2003-12-03,ERP,a00
6,7,10.1093/brain/awae292,2024-12-03,tDCS,a00
7,8,10.1162/jocn_a_02070,2024-01-01,SSVEP,a00


Now, the result of concatenating the values of "partial_id" and "study_id" should produce string-type values. However, the "study_id" column expects to receive, even when its values are modified, INTEGER data types.
This means that the data type of the column would need to be changed—something that cannot be easily done in this case, since it has a UNIQUE or PRIMARY KEY constraint. Therefore, the constraint must first be removed.
This operation is [supported by PostgreSQL](https://neon.tech/postgresql/postgresql-tutorial/postgresql-primary-key#drop-a-primary-key), but not by DuckDB, which responded as follows to our attempt to execute a `DROP CONSTRAINT`:

In [ ]:
# If we had set study_id_as_pkey (instead of commenting that out)
# %%sql
# ALTER TABLE study DROP CONSTRAINT study_id_as_pkey;
# -- This block of code is therefore "useless" here (DuckDB), but accepted by PostgreSQL (we used DB Fiddle to check)

In any case, the next step would have been this (and it would have produced the desired result: we made further attempts by creating the table without specifying the primary key in order to bypass the constraint issue):

In [ ]:
# Change of the data type of study_id and concatenation of the values of partial_id with study_id
%%sql
ALTER TABLE study ALTER study_id SET DATA TYPE VARCHAR USING CONCAT(partial_id, study_id);

,Success


In [ ]:
# Now we don't need the partial_id anymore, so we can drop the column
%%sql
ALTER TABLE study DROP partial_id;

,Success


In [ ]:
# The result
%%sql
SELECT *
FROM study;

,study_id,doi,publication_date,technique_abbreviation
0,a001,10.1073/pnas.1015950108,2011-02-15,MRI
1,a002,10.1073/pnas.2309975121,2024-02-28,fMRI
2,a003,10.1016/j.cobeha.2015.06.003,2015-06-03,tDCS
3,a004,10.1126/science.1093065,2004-02-20,fMRI
4,a005,10.1073/pnas.2405850121,2024-03-22,MRI
5,a006,10.3758/cabn.3.4.255,2003-12-03,ERP
6,a007,10.1093/brain/awae292,2024-12-03,tDCS
7,a008,10.1162/jocn_a_02070,2024-01-01,SSVEP


However, this method has some non-negligible drawbacks:
* It requires great care when changing the data type of a primary key
* More importantly, you cannot apply `DROP CONSTRAINT` to a primary key if it is already referenced as a foreign key in other relations

Additionally, this method doesn't allow, in the case of an id > 9, for selecting which tuples should have a different prefix like "a0" (to get "a010" instead of "a0010", for example).

An operation that does allow modification of a tuple or cell under specific conditions like the one mentioned above is `UPDATE`, which falls under DML operations.

With `UPDATE`, you would start from the original table, without adding new columns, but it would still be necessary to change the data type of "study_id" from INTEGER to VARCHAR.
In both cases, we are aware of the possible implications of this process—specifically, changing the data type of a surrogate key that belongs to a numerical sequence:
How do you handle new studies that are added after such a change?
You would need to restart the sequence from the last number, but in a new temporary table to then join back with the original one?

In [ ]:
#%%sql
#ALTER TABLE study DROP CONSTRAINT study_id_as_pkey; -- In PostgreSQL
#-- As mentioned before, this block of code won't work in DuckDB, because this PK is a FK in other relations

In [ ]:
# Here we change the data type of study_id as well
%%sql
ALTER TABLE study ALTER study_id SET DATA TYPE VARCHAR;

,Success


In [ ]:
# Afterwards, we specify which strings to concatenate, and which conditions they must respect
%%sql
UPDATE study
SET study_id = CONCAT('a00', study_id)
WHERE study_id ~ '^[1-9]$'

,Success


In [ ]:
# The final result
%%sql
SELECT *
FROM study;

,study_id,doi,publication_date,technique_abbreviation
0,a001,10.1073/pnas.1015950108,2011-02-15,MRI
1,a002,10.1073/pnas.2309975121,2024-02-28,fMRI
2,a003,10.1016/j.cobeha.2015.06.003,2015-06-03,tDCS
3,a004,10.1126/science.1093065,2004-02-20,fMRI
4,a005,10.1073/pnas.2405850121,2024-03-22,MRI
5,a006,10.3758/cabn.3.4.255,2003-12-03,ERP
6,a007,10.1093/brain/awae292,2024-12-03,tDCS
7,a008,10.1162/jocn_a_02070,2024-01-01,SSVEP


Since "study_id" is a foreign key in other relations, this change in the key should propagate to those relations as well, to avoid errors.
Taking the "cerebral_function" relation as an example, the CREATE TABLE (following [PostgreSQL documentation](https://www.postgresql.org/docs/current/ddl-constraints.html#DDL-CONSTRAINTS-FK), which DuckDB's dialect is based on) should be structured as follows:

> *Note: these considerations only apply in situations where, unlike in our case, the data type of the PK does not need to be changed via DROP CONSTRAINT, but only the value of the key needs modification.*



In [ ]:
# OPTION 1
#%%sql
#CREATE TABLE cerebral_function (
  #functional_macroarea varchar(20) PRIMARY KEY,
  #specification varchar(30) UNIQUE NOT NULL,
  #brain_area varchar(30) UNIQUE NOT NULL,
  #study_id INTEGER,
  #CONSTRAINT fk_study_id_in_funzione
    #FOREIGN KEY(study_id)
      #REFERENCES study(study_id) ON UPDATE CASCADE ON DELETE CASCADE
#);

In [ ]:
# OPTION 2
#%%sql
#CREATE TABLE cerebral_function (
    #study_id INTEGER REFERENCES study.study_id ON UPDATE CASCADE ON DELETE CASCADE,
    #functional_macroarea varchar(20),
    #specification varchar(30) UNIQUE NOT NULL,
    #brain_area varchar(30) UNIQUE NOT NULL
    #PRIMARY KEY (study_id, functional_macroarea, specification, brain_area)
#);

The code is commented out because [DuckDB has limitations](https://duckdb.org/docs/sql/statements/create_table.html#limitations) when it comes to cascading behavior.
In fact, even though this wasn't an ON DELETE case like in the linked example, the error message we encountered when trying to run the code was the following:


```
RuntimeError: (duckdb.duckdb.ParserException) Parser Error: FOREIGN KEY constraints cannot use CASCADE, SET NULL or SET DEFAULT
```



In conclusion, what we attempted here ultimately did not serve our goals effectively. Still, it allowed us to explore and test other examples of limitations imposed by primary key constraints and the need to maintain referential integrity between relations that share attributes as foreign keys.

Moreover, this process allowed us to compare PostgreSQL (whose conventions, as mentioned, are mostly followed by DuckDB) with DuckDB, which turned out to be user-friendly in the querying phase through Excel tables, but quite limited in performing other operations, as emphasized in the commented code blocks of this section.